# Árboles de decisión y redes Bayesianas para predecir el ganador de un duelo en el juego League of Legends

In [1437]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score

In [1438]:
data = pd.read_csv("games.csv", sep=";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50286 entries, 0 to 50285
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gameDuration        50286 non-null  int64
 1   firstBlood          50286 non-null  int64
 2   firstTower          50286 non-null  int64
 3   firstInhibitor      50286 non-null  int64
 4   firstBaron          50286 non-null  int64
 5   firstDragon         50286 non-null  int64
 6   firstRiftHerald     50286 non-null  int64
 7   t1_towerKills       50286 non-null  int64
 8   t1_inhibitorKills   50286 non-null  int64
 9   t1_baronKills       50286 non-null  int64
 10  t1_dragonKills      50286 non-null  int64
 11  t1_riftHeraldKills  50286 non-null  int64
 12  t2_towerKills       50286 non-null  int64
 13  t2_inhibitorKills   50286 non-null  int64
 14  t2_baronKills       50286 non-null  int64
 15  t2_dragonKills      50286 non-null  int64
 16  t2_riftHeraldKills  50286 non-null  int6

In [1439]:
data.head()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,winner
0,1949,2,1,1,1,1,2,11,1,2,3,0,5,0,0,1,1,1
1,1851,1,1,1,0,1,1,10,4,0,2,1,2,0,0,0,0,1
2,1493,2,1,1,1,2,0,8,1,1,1,0,2,0,0,1,0,1
3,1758,1,1,1,1,1,0,9,2,1,2,0,0,0,0,0,0,1
4,2094,2,1,1,1,1,0,9,2,1,3,0,3,0,0,1,0,1


# Descripción estadística

In [1440]:
data.describe()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,winner
count,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000,50286.000000
mean,1871.386032,1.487233,1.485125,1.339578,0.948693,1.477270,0.749195,5.835163,1.041781,0.381200,1.420356,0.257487,5.681999,1.008611,0.424472,1.437975,0.245854,1.493776
std,450.876052,0.499842,0.500499,0.652960,0.838988,0.530346,0.824393,3.740575,1.269064,0.588001,1.201740,0.437254,3.809336,1.261867,0.617672,1.219412,0.430597,0.499966
min,610.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1554.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1846.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000,0.000000,1.000000,0.000000,6.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,2156.750000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,9.000000,2.000000,1.000000,2.000000,1.000000,9.000000,2.000000,1.000000,2.000000,0.000000,2.000000
max,4728.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,11.000000,10.000000,5.000000,6.000000,1.000000,11.000000,10.000000,4.000000,6.000000,1.000000,2.000000


# Se separan los datos (80% de entrenamiento y 20% de prueba)

In [1441]:
from sklearn.model_selection import train_test_split 
N=len(data)
cTrain=int(N*0.8) # 80% para entrenar y 20% para probar
cTest=N-cTrain
print(N,cTrain,cTest)
train_data,test_data= sklearn.model_selection.train_test_split(data, train_size=cTrain, test_size=cTest)

50286 40228 10058


In [1442]:
train_data.shape

(40228, 18)

In [1443]:
train_data.head()

,gameDuration,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,winner
50117,1830,2,1,2,2,2,0,4,0,0,0,0,11,3,1,4,0,2
19779,1241,1,1,0,0,1,0,3,0,0,1,0,1,0,0,0,0,1
29407,1975,2,2,2,0,1,1,1,0,0,2,1,10,3,0,1,0,2
29001,2245,1,1,1,1,2,0,10,1,1,1,0,3,0,0,3,0,1
2668,2028,1,2,1,2,1,0,7,1,0,1,0,7,1,1,2,0,2


# Se hace un Pipeline para todos los atributos, teniendo en cuenta que todos son de tipo entero


In [1444]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_attribs = ['gameDuration', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills', 't1_dragonKills', 't1_riftHeraldKills', 't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills', 't2_riftHeraldKills']
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()) 
    
    ])

In [1445]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
])

In [1446]:
X_train = full_pipeline.fit_transform(train_data)

X_train[0,:]

array([-0.0897241 ,  1.02430849, -0.97132954,  1.01108766,  1.25387091,
        0.98258665, -0.90847903, -0.48738421, -0.81981077, -0.64817905,
       -1.18102266, -0.58753137,  1.39231218,  1.57406628,  0.93225697,
        2.09669829, -0.57187727])

# Se extrae las etiquetas de clase

In [1447]:
y_train = train_data['winner']
y_train

50117    2
19779    1
29407    2
29001    1
2668     2
        ..
46233    1
41740    1
37252    1
38387    2
40762    2
Name: winner, Length: 40228, dtype: int64

# Se crean los áboles de decisión


# Hiperparámetros criterion=gini, splitter=best, y random_state=123.

In [1529]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

modelos=np.array([None]*10)
j=0
for i in range (100, 1100, 100):
    modelos[j]= tree.DecisionTreeClassifier(criterion='gini', max_depth=i, splitter='best', random_state=123)
    modelos[j].fit(X_train, y_train)
    scores = cross_val_score(modelos[j], X_train, y_train, cv=5,scoring='accuracy')
    print(scores)
    print(scores.mean())
    j+=1

#tree.plot_tree(modelo1)
#tree.export_graphviz(decision_tree=modelo1, class_names=True, out_file='Arbol league of legends1.dot')

[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547
[0.96979866 0.97004723 0.96905295 0.96842759 0.97004351]
0.9694739863559547


In [1526]:
X_test = full_pipeline.transform(test_data)
X_test

array([[ 0.93667216,  1.02430849,  1.02604533, ...,  0.93225697,
         1.27766216, -0.57187727],
       [-0.20278287,  1.02430849,  1.02604533, ..., -0.68710627,
        -0.3604101 , -0.57187727],
       [-0.83458187,  1.02430849, -0.97132954, ..., -0.68710627,
        -1.17944623, -0.57187727],
       ...,
       [ 0.19624808,  1.02430849, -0.97132954, ...,  0.93225697,
        -0.3604101 ,  1.74862694],
       [ 0.83691443, -0.97626839, -0.97132954, ..., -0.68710627,
        -1.17944623,  1.74862694],
       [-0.12519352,  1.02430849,  1.02604533, ..., -0.68710627,
         0.45862603, -0.57187727]])

In [1530]:
y_pred=np.array([None]*10)

for i in range (10):

    y_pred[i] = modelos[i].predict(X_test) 
    print(y_pred[i])

[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]


In [1532]:
y_test = test_data["winner"]
y_test[:10]

24630    2
17053    1
11482    1
45178    1
24277    1
4181     1
16009    2
46424    1
43628    1
39499    2
Name: winner, dtype: int64

In [1539]:
precision=[]
for i in range (10):
    precision.append([f"modelo {i+1}",accuracy_score(y_test, y_pred[i])])
    #print(precision[i])
tabla_precision=pd.DataFrame(precision, columns=["Modelo", "Precisión"])
tabla_precision


,Modelo,Precisión
0,modelo 1,0.975442
1,modelo 2,0.975442
2,modelo 3,0.975442
3,modelo 4,0.975442
4,modelo 5,0.975442
5,modelo 6,0.975442
6,modelo 7,0.975442
7,modelo 8,0.975442
8,modelo 9,0.975442
9,modelo 10,0.975442


# Hiperparámetros criterion=entropy, splitter=best, y random_state=123.

In [1558]:
modelos_eb=np.array([None]*10)
j=0
for i in range (100, 1100, 100):
    modelos_eb[j]= tree.DecisionTreeClassifier(criterion='entropy', max_depth=i, splitter='best', random_state=123)
    modelos_eb[j].fit(X_train, y_train)
    scores = cross_val_score(modelos_eb[j], X_train, y_train, cv=5,scoring='accuracy')
    print(scores)
    print(scores.mean())
    j+=1

#tree.plot_tree(modelo11)
#tree.export_graphviz(decision_tree=modelo11, class_names=True, out_file='Arbol league of legends11.dot')

[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097
[0.97129008 0.97141437 0.97129008 0.97165942 0.97041641]
0.9712140709874097


In [1555]:
y_pred_eb=np.array([None]*10)

for i in range (10):

    y_pred_eb[i] = modelos_eb[i].predict(X_test) 
    print(y_pred_eb[i])

[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]
[2 1 1 ... 2 1 2]


In [1556]:
precision_eb=[]
for i in range (10):
    precision_eb.append([f"modelo {i+1}",accuracy_score(y_test, y_pred_eb[i])])
    #print(precision[i])
tabla_precision_eb=pd.DataFrame(precision_eb, columns=["Modelo", "Precisión"])
tabla_precision_eb


,Modelo,Precisión
0,modelo 1,0.975045
1,modelo 2,0.975045
2,modelo 3,0.975045
3,modelo 4,0.975045
4,modelo 5,0.975045
5,modelo 6,0.975045
6,modelo 7,0.975045
7,modelo 8,0.975045
8,modelo 9,0.975045
9,modelo 10,0.975045


# Hiperparámetros criterion=entropy, splitter=random, y random_state=123. 

In [1557]:
modelos_er=np.array([None]*10)
j=0
for i in range (100, 1100, 100):
    modelos_er[j]= tree.DecisionTreeClassifier(criterion='entropy', max_depth=i, splitter='random', random_state=123)
    modelos_er[j].fit(X_train, y_train)
    scores = cross_val_score(modelos_er[j], X_train, y_train, cv=5,scoring='accuracy')
    print(scores)
    print(scores.mean())
    j+=1

#tree.plot_tree(modelo21)
#tree.export_graphviz(decision_tree=modelo21, class_names=True, out_file='Arbol league of legends21.dot')

[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516
[0.97004723 0.96706438 0.97278151 0.97016781 0.97016781]
0.970045745354516


In [1559]:
y_pred_er=np.array([None]*10)

for i in range (10):

    y_pred_er[i] = modelos_er[i].predict(X_test) 
    print(y_pred_er[i])

[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]
[2 1 1 ... 2 2 2]


In [1560]:
precision_er=[]
for i in range (10):
    precision_er.append([f"modelo {i+1}",accuracy_score(y_test, y_pred_er[i])])
    #print(precision[i])
tabla_precision_er=pd.DataFrame(precision_er, columns=["Modelo", "Precisión"])
tabla_precision_er

,Modelo,Precisión
0,modelo 1,0.973255
1,modelo 2,0.973255
2,modelo 3,0.973255
3,modelo 4,0.973255
4,modelo 5,0.973255
5,modelo 6,0.973255
6,modelo 7,0.973255
7,modelo 8,0.973255
8,modelo 9,0.973255
9,modelo 10,0.973255
